In [28]:
import sys, os

In [29]:
sys.path.append(os.path.join(os.path.dirname(sys.path[0]),'../spectral-tree-inference/spectraltree'))

In [30]:
import numpy as np
import utils
import generation
import reconstruct_tree
import dendropy
import scipy
from itertools import product

from dendropy.model.discrete import simulate_discrete_chars, Jc69, Hky85
from dendropy.calculate.treecompare import symmetric_difference

## Example for running spectral neighbor joining

In [5]:
N = 400 # length of sequences
num_taxa = 32 # number of taxa
jc = generation.Jukes_Cantor() # transition matrix object
mutation_rate = [jc.p2t(0.95)]

In [6]:
reference_tree = utils.unrooted_birth_death_tree(num_taxa, birth_rate=1)

In [7]:
for x in reference_tree.preorder_edge_iter():
    x.length = 1

In [8]:
observations = generation.simulate_sequences_ordered(N, tree_model=reference_tree, seq_model=jc, mutation_rate=mutation_rate)

In [9]:
observations.shape

(32, 400)

In [10]:
# spectral neighbor joining
snj = reconstruct_tree.SpectralNeighborJoining(reconstruct_tree.JC_similarity_matrix)
tree_rec = snj(observations, reference_tree.taxon_namespace)
RF,F1 = reconstruct_tree.compare_trees(tree_rec, reference_tree)
print("SNJ: ")
print("RF = ",RF)
print("F1% = ",F1)
print("")

In [12]:
# neighbor joining 
nj = reconstruct_tree.NeighborJoining(reconstruct_tree.JC_similarity_matrix)
tree_rec = nj(observations,reference_tree.taxon_namespace)
RF,F1 = reconstruct_tree.compare_trees(tree_rec, reference_tree)
print("")
print("NJ: ")
print("RF = ",RF)
print("F1% = ",F1)
print("")


NJ: 
RF =  0
F1% =  100.0



## Example using user-supplied tree file

In [6]:
tree_path = "/home/mw957/project/repos/spec_tree/data/skygrid_J2.newick"
fasta_path = "/home/mw957/project/repos/spec_tree/data/H3N2_NewYork.fasta"

In [7]:
H3N2_tree = dendropy.Tree.get(path=tree_path, schema="newick")
H3N2_dna = dendropy.DnaCharacterMatrix.get(file=open(fasta_path, "r"), schema="fasta")

In [71]:
ch_list = list()
for t in H3N2_dna.taxon_namespace:
    ch_list.append([x.symbol for x in H3N2_dna[t]])

leafs_idx = [i.label[0] != " " for i in H3N2_dna.taxon_namespace]

In [72]:
np.unique(ch_list)
# https://www.bioinformatics.org/sms/iupac.html
# H: A or C or T
# K: G or T
# M: A or C
# N: any base
# R: A or G
# W: A or T
# Y: C or T

array(['-', 'A', 'C', 'G', 'H', 'K', 'M', 'N', 'R', 'T', 'W', 'Y'],
      dtype='<U1')

In [10]:
ch_list_num = np.array(ch_list)
ch_list_num = ch_list_num[leafs_idx]
ch_list_num = np.where(ch_list_num=='A', 1, ch_list_num)
ch_list_num = np.where(ch_list_num=='C', 2, ch_list_num)
ch_list_num = np.where(ch_list_num=='G', 3, ch_list_num)
ch_list_num = np.where(ch_list_num=='T', 4, ch_list_num)
ch_list_num = np.where(np.isin(ch_list_num, ['-', "H", "K", "M", "N", "R", "W", "Y"]), 
                       -1, ch_list_num)
ch_list_num = ch_list_num.astype('int')

In [11]:
ch_list_num.shape

(565, 1737)

In [12]:
def hamming_dist_missing_values(vals, missing_val =0):
    hamming_matrix = scipy.spatial.distance.squareform(scipy.spatial.distance.pdist(vals, metric='hamming'))
    missing_array = (vals==missing_val)
    pdist_xor = scipy.spatial.distance.squareform(scipy.spatial.distance.pdist(missing_array, lambda u,v: np.sum(np.logical_xor(u,v))))
    pdist_or = scipy.spatial.distance.squareform(scipy.spatial.distance.pdist(missing_array, lambda u,v: np.sum(np.logical_or(u,v))))
    return (hamming_matrix*vals.shape[1] - pdist_xor) / (np.ones_like(hamming_matrix) * vals.shape[1] - pdist_or)

def JC_similarity_matrix_missing_values(observations, classes=None):
    assert classes is None
    if classes is None:
        classes = np.unique(observations)
    k = len(classes)
    hamming_matrix_corrected = hamming_dist_missing_values(observations)
    inside_log = 1 - hamming_matrix_corrected*k/(k-1)
    return inside_log**(k-1)

nj = reconstruct_tree.NeighborJoining(JC_similarity_matrix_missing_values)
tree_rec = nj(ch_list_num, H3N2_tree.taxon_namespace)

## Simulate sequences from the tree

In [8]:
N = 1000

In [9]:
# simulate sequences from the HKY model (set transition/transversion bias = 2)
data_HKY = simulate_discrete_chars(N, H3N2_tree, Hky85(kappa = 2), mutation_rate=0.1)
ch_list = list()
for t in data_HKY.taxon_namespace:
    ch_list.append([x.symbol for x in data_HKY[t]])

leafs_idx = [i.label[0] != " " for i in data_HKY.taxon_namespace]
ch_arr = np.array(ch_list)
ch_arr = ch_arr[leafs_idx]

In [16]:
identical = [np.mean(a == b) for a, b in product(ch_arr, repeat = 2)]

In [21]:
identical = np.array(identical)

In [56]:
print(np.mean(identical[identical != 1]), np.median(identical[identical != 1]), np.min(identical[identical != 1]), np.max(identical[identical != 1]))

0.6758710513597821 0.682 0.408 0.999


In [147]:
raxml_HKY = reconstruct_tree.RAxML()
import time
start_time = time.time()
raxml_HKY_tree = raxml_HKY(data_HKY, raxml_args="-T 2 --HKY85 -c 1")
print("--- %s seconds ---" % (time.time() - start_time))

RF,F1 = reconstruct_tree.compare_trees(raxml_HKY_tree, H3N2_tree)
print("")
print("HKY: ")
print("RF = ",RF)
print("F1% = ",F1)
print("")

####
# python scripts/hky_raxml.py

# NJ:
# RF =  162
# F1% =  92.81914893617022

--- 464.3687424659729 seconds ---

HKY: 
RF =  108
F1% =  95.21276595744682



In [63]:
## compute the similarity matrix under HKY model
def HKY_similarity_matrix(observations, classes=None, debug = False):
    m, N = observations.shape
    if classes is None:
        classes = np.unique(observations)
    k = len(classes)
    # From Tamura, K., and M. Nei. 1993
    # for each pair of sequences, 
    # 1. estimate the base frequency
    # 2. compute purine transition proportion P1 (A <-> G)
    # 3. compute pyrimidine transition proportion P2 (T <-> C)
    # 3. compute transversion proportion Q (A <-> C, A <-> T, G <-> C, G <-> T)
    
    # average base frequency for pairs of sequences
    print("Computing the average base frequency for each possible pairs...")
    g = {}
    for x in classes:
        obs_x = observations == x
        g[x] = np.array([np.mean(np.hstack([a, b])) for a, b in product(obs_x, repeat = 2)]).reshape((m, m))
    
    g["R"] = g["A"] + g["G"]
    g["Y"] = g["T"] + g["C"]
    
    # compute counts of differences
    print("Computing the counts for differences for each possible pairs...")
    P = {}
    for i, x in enumerate(classes):
        other_classes = np.delete(classes, i)
        for y in other_classes:
            P_x_y = np.array([np.mean(np.logical_and(a == x, b == y)) for a, b in product(observations, repeat = 2)]).reshape((m, m))
            P[x + y] = P_x_y
            
    P_1 = P['AG'] + P["GA"]
    P_2 = P['CT'] + P['TC']
    Q = P['AC'] + P['CA'] + P['AT'] + P['TA'] +\
        P['GC'] + P['CG'] + P['GT'] + P['TG']

    # compute the similarity 
    print("Computing similarity matrix")
    R = (1 - g["R"]/(2 * g["A"] * g["G"]) * P_1 - 1 / (2 * g["R"]) * Q)
    Y = (1 - g["Y"]/(2 * g["T"] * g["C"]) * P_2 - 1 / (2 * g["Y"]) * Q)
    T = (1 - 1/(2 * g["R"] * g["Y"]) * Q)
    S = np.sign(R) * (np.abs(R))**(2 * g["A"] * g["G"] / g["R"])
    S += np.sign(Y) * (np.abs(Y))**(2 * g["T"] * g["C"] / g["Y"])
    S += np.sign(T) * (np.abs(T))**(2 * (g["R"] * g["Y"] - g["A"] * g["G"] * g["Y"] / g["R"] - g["T"] * g["C"] * g["R"] / g["Y"]))
    
    if debug:
        return (P_1, P_2, Q, R, Y, T, S)
    else: 
        return S

In [133]:
# spectral neighbor joining
snj = reconstruct_tree.SpectralNeighborJoining(HKY_similarity_matrix)

In [141]:
import time
start_time = time.time()
tree_rec = snj(ch_arr, H3N2_tree.taxon_namespace)

print("--- %s seconds ---" % (time.time() - start_time))

Computing the average base frequency for each possible pairs...
Computing the counts for differences for each possible pairs...
Computing similarity matrix
--- 210.27279615402222 seconds ---


In [142]:
RF,F1 = reconstruct_tree.compare_trees(tree_rec, H3N2_tree)
print("SNJ: ")
print("RF = ", RF)
print("F1% = ", F1)
print("")

SNJ: 
RF =  570
F1% =  74.7340425531915



### Neighbor joining

In [129]:
nj = reconstruct_tree.NeighborJoining(HKY_similarity_matrix)

In [143]:
import time
start_time = time.time()
tree_rec = nj(ch_arr, H3N2_tree.taxon_namespace)
print("--- %s seconds ---" % (time.time() - start_time))

Computing the average base frequency for each possible pairs...
Computing the counts for differences for each possible pairs...
Computing similarity matrix
--- 205.862291097641 seconds ---


In [144]:
RF,F1 = reconstruct_tree.compare_trees(tree_rec, H3N2_tree)
print("")
print("NJ: ")
print("RF = ",RF)
print("F1% = ",F1)
print("")


NJ: 
RF =  166
F1% =  92.64184397163122



### Deep spectal neighbor joining 

In [38]:
spectral_method = reconstruct_tree.SpectralTreeReconstruction(reconstruct_tree.NeighborJoining,
                                                              HKY_similarity_matrix)

In [39]:
import time
start_time = time.time()
tree_rec = spectral_method.deep_spectral_tree_reonstruction(ch_arr, HKY_similarity_matrix, 
                                                            taxon_namespace = H3N2_tree.taxon_namespace, 
                                                            threshhold = 35)
print("--- %s seconds ---" % (time.time() - start_time))

Computing the average base frequency for each possible pairs...
Computing the counts for differences for each possible pairs...
Computing similarity matrix
--- 209.564204454422 seconds ---


In [40]:
RF,F1 = reconstruct_tree.compare_trees(tree_rec, H3N2_tree)
print("Spectral deep (35): ")
print("RF = ", RF)
print("F1% = ", F1)
print("")

Spectral deep (35): 
RF =  172
F1% =  92.3758865248227



In [33]:
# 2. Spectral deep with NJ as the sub method with a threshold of 128
spectral_method = reconstruct_tree.SpectralTreeReconstruction(reconstruct_tree.NeighborJoining,
                                                              HKY_similarity_matrix)
import time
start_time = time.time()
tree_rec = spectral_method.deep_spectral_tree_reonstruction(ch_arr, HKY_similarity_matrix, 
                                                            taxon_namespace = H3N2_tree.taxon_namespace, 
                                                            threshhold = 128)
print("--- %s seconds ---" % (time.time() - start_time))

Computing the average base frequency for each possible pairs...
Computing the counts for differences for each possible pairs...
Computing similarity matrix
--- 197.9316599369049 seconds ---


In [34]:
RF,F1 = reconstruct_tree.compare_trees(tree_rec, H3N2_tree)
print("Spectral deep (128): ")
print("RF = ", RF)
print("F1% = ", F1)
print("")

Spectral deep (128): 
RF =  148
F1% =  93.43971631205673



In [45]:
# 4. Spectral deep with NJ as the sub method with a threshold of 256
spectral_method = reconstruct_tree.SpectralTreeReconstruction(reconstruct_tree.NeighborJoining,
                                                              HKY_similarity_matrix)
import time
start_time = time.time()
tree_rec = spectral_method.deep_spectral_tree_reonstruction(ch_arr, HKY_similarity_matrix, 
                                                            taxon_namespace = H3N2_tree.taxon_namespace, 
                                                            threshhold = 256)
print("--- %s seconds ---" % (time.time() - start_time))

Computing the average base frequency for each possible pairs...
Computing the counts for differences for each possible pairs...
Computing similarity matrix
--- 189.9345338344574 seconds ---


In [47]:
RF,F1 = reconstruct_tree.compare_trees(tree_rec, H3N2_tree)
print("Spectral deep (256): ")
print("RF = ", RF)
print("F1% = ", F1)
print("")

Spectral deep (256): 
RF =  158
F1% =  92.99645390070923



## Summary

In [49]:
import pandas as pd
pd.DataFrame({'method': ["RaxML", "NJ", "Spectral deep (NJ): 35", "Spectral deep (NJ): 128", "Spectral deep (NJ): 256", "SNJ"],
             'run time': [ 464.36, 205.86, 215.95, 197.93, 189.93, 210.27],
             'RF': [108, 166, 202, 148, 158, 570],
             'F1 %': [95.21, 92.64, 91.05, 93.44, 93.00, 74.73]}).sort_values("F1 %", ascending = False)

,method,run time,RF,F1 %
0,RaxML,464.36,108,95.21
3,Spectral deep (NJ): 128,197.93,148,93.44
4,Spectral deep (NJ): 256,189.93,158,93.00
1,NJ,205.86,166,92.64
2,Spectral deep (NJ): 35,215.95,202,91.05
5,SNJ,210.27,570,74.73


### Try to use the data directly

#### neighbor joining

In [64]:
def HKY_similarity_matrix_ATCG(observations):
    return HKY_similarity_matrix(observations, classes=["A", "T", "C", "G"], debug = False)
    
nj = reconstruct_tree.NeighborJoining(HKY_similarity_matrix_ATCG)

In [65]:
import time
start_time = time.time()
tree_rec = nj(np.array(ch_list), H3N2_tree.taxon_namespace)
print("--- %s seconds ---" % (time.time() - start_time))

Computing the average base frequency for each possible pairs...
Computing the counts for differences for each possible pairs...
Computing similarity matrix
--- 203.94027876853943 seconds ---


In [66]:
RF,F1 = reconstruct_tree.compare_trees(tree_rec, H3N2_tree)
print("")
print("NJ: ")
print("RF = ",RF)
print("F1% = ",F1)
print("")


NJ: 
RF =  196
F1% =  91.31205673758865



In [73]:
# 2. Spectral deep with NJ as the sub method with a threshold of 128
spectral_method = reconstruct_tree.SpectralTreeReconstruction(reconstruct_tree.NeighborJoining,
                                                              HKY_similarity_matrix_ATCG)
import time
start_time = time.time()
tree_rec = spectral_method.deep_spectral_tree_reonstruction(np.array(ch_list), HKY_similarity_matrix_ATCG, 
                                                            taxon_namespace = H3N2_tree.taxon_namespace, 
                                                            threshhold = 128)
print("--- %s seconds ---" % (time.time() - start_time))

Computing the average base frequency for each possible pairs...
Computing the counts for differences for each possible pairs...
Computing similarity matrix


/home/mw957/project/conda_envs/r_env/lib/python3.7/site-packages/sklearn/decomposition/_truncated_svd.py:194: RuntimeWarning: divide by zero encountered in true_divide
  self.explained_variance_ratio_ = exp_var / full_var
/home/mw957/project/conda_envs/r_env/lib/python3.7/site-packages/sklearn/decomposition/_truncated_svd.py:194: RuntimeWarning: divide by zero encountered in true_divide
  self.explained_variance_ratio_ = exp_var / full_var
/home/mw957/project/conda_envs/r_env/lib/python3.7/site-packages/sklearn/decomposition/_truncated_svd.py:194: RuntimeWarning: divide by zero encountered in true_divide
  self.explained_variance_ratio_ = exp_var / full_var
/home/mw957/project/conda_envs/r_env/lib/python3.7/site-packages/sklearn/decomposition/_truncated_svd.py:194: RuntimeWarning: divide by zero encountered in true_divide
  self.explained_variance_ratio_ = exp_var / full_var
/home/mw957/project/conda_envs/r_env/lib/python3.7/site-packages/sklearn/decomposition/_truncated_svd.py:194: Ru

--- 275.51211047172546 seconds ---


In [74]:
RF,F1 = reconstruct_tree.compare_trees(tree_rec, H3N2_tree)
print("Spectral deep (128): ")
print("RF = ", RF)
print("F1% = ", F1)
print("")

Spectral deep (128): 
RF =  1116
F1% =  50.53191489361703



In [69]:
# 3. RaXML
raxml_HKY = reconstruct_tree.RAxML()
import time
start_time = time.time()
raxml_HKY_tree = raxml_HKY(np.array(ch_list), raxml_args="-T 2 --HKY85 -c 1")
print("--- %s seconds ---" % (time.time() - start_time))

RF,F1 = reconstruct_tree.compare_trees(raxml_HKY_tree, H3N2_tree)
print("")
print("RaXML: ")
print("RF = ",RF)
print("F1% = ",F1)
print("")

TypeError: list indices must be integers or slices, not numpy.str_